# Despliegue de modelos

In [6]:
import gradio as gr
from keras.models import load_model
import gdown  
import os
import tensorflow as tf
import numpy as np
import cv2


#verificar si existee modelo.h5
output = 'modelo.h5'

if os.path.isfile(output):
    print("El modelo ya existe")
else:
    print("El modelo no existe, se descargara")
    url = 'https://drive.google.com/uc?id=1_AghpfVDq0o6xnDdnjrySUSBsB5Qkq04'

    gdown.download(url, output, quiet=False)


modelo = load_model(output)  



val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_folder = "../data_acquisition/test"


image_size = (250, 250)
batch_size = 16

validation_generator = val_datagen.flow_from_directory(directory = test_folder + "/",
                                                       target_size = image_size,
                                                       batch_size = batch_size,
                                                       class_mode = 'categorical')


def mapeo_indices(listado):

    listado_clases = list(validation_generator.class_indices.keys())

    true_index = np.argmax(listado,axis=1)
    salida_real = [listado_clases[i] for i in true_index]

    return salida_real




def predict(img):
    
    if img.shape[0] >250 or img.shape[1] >250:
            
        height= img.shape[0]
        width= img.shape[1]
        
        c_height= round(height/2)
        c_width= round(width/2)

        img = img[c_height-125:c_height+125, c_width-125:c_width+125]

    
    img = img / 255.0
    img=img.reshape((-1, 250, 250, 3))

    pred= modelo.predict(img)
    
    pred=mapeo_indices(pred)
        
    initial_path="../data_acquisition/train"
    var=pred[0]

    ruta=os.path.join(initial_path,var )

    imagen_de_base=os.path.join(ruta,np.random.choice(os.listdir(ruta)) )
    comparativo=cv2.imread(imagen_de_base)
    comparativo = cv2.cvtColor(comparativo, cv2.COLOR_BGR2RGB)
    


    return pred[0],comparativo



with gr.Blocks(theme=gr.themes.Soft()) as demo:

    gr.Markdown("## Reconocimiento de personas basado en imágenes previas")
    gr.Markdown("## Andrés Forero")
    


    input = gr.Image(label="Sube una nueva imagen de una persona de la base de datos")
    b_predict = gr.Button("Predecir")
    with gr.Row():
            
        output = gr.Textbox( label="Predicción de etiqueta",scale=1)
        output2 = gr.Image( label="Foto aleatorio de predicción dentro de base de datos",scale=4,width=250,height=250)

    b_predict.click(predict, inputs=[input], outputs=[output,output2])


demo.launch()




El modelo ya existe
Found 3828 images belonging to 1680 classes.
Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 0s 31ms/step
